In [23]:
import torch

In [1]:
with open("data.txt" , 'r',encoding='utf-8') as f:
    text = f.read()

In [2]:
print("length of dataset in characters is :: ", len(text))

length of dataset in characters is ::  1115394


In [6]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [7]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [19]:
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda s: (''.join([itos[c] for c in s]))

In [22]:
print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [24]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)

torch.Size([1115394]) torch.int64


In [26]:
# train and validation split
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [46]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    # generate a small size batches of inputs x and target y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print("inputs")
print(xb.shape)
print(xb)

print("targets")
print(yb.shape)
print(yb)

for b in range(batch_size):
    for t in range(block_size):
        contex = xb[b,:t+1]
        target = yb[b,t]
        print(f"when input is {contex} the target: {target}")        

inputs
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
when input is tensor([24]) the target: 43
when input is tensor([24, 43]) the target: 58
when input is tensor([24, 43, 58]) the target: 5
when input is tensor([24, 43, 58,  5]) the target: 57
when input is tensor([24, 43, 58,  5, 57]) the target: 1
when input is tensor([24, 43, 58,  5, 57,  1]) the target: 46
when input is tensor([24, 43, 58,  5, 57,  1, 46]) the target: 43
when input is tensor([24, 43, 58,  5, 57,  1, 46, 43]) the target: 39
when input is tensor([44]) the target: 53
when input is tensor([44, 53]) the target: 56
when input is tensor([44, 53, 56]) the target: 1
when input is tensor([44, 

In [81]:
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)


class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx and target are B, T tensor
        logits = self.token_embedding_table(idx)  #B, T, C

        if targets is None:
            loss = 0
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indecis of current contex
        for _ in range(max_new_tokens):
            logits, loss = self(idx)  #call forward and get prediction
            logits = logits[:,-1, :] #becomes (B, C)
            probs = F.softmax(logits, dim=1) #softmax to get probablities
            idx_next = torch.multinomial(probs, num_samples=1)  #(B, 1)
            idx = torch.cat((idx, idx_next), dim=1)  #(B, T+1)
        return idx
            
            

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)
print(decode(m.generate(idx = torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [99]:
# creating python optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [107]:
batch_size = 32
for steps in range(10000):
    # sample a batch of data
    xb, yb = get_batch('train')

    # eval the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.5332515239715576


In [115]:
print(decode(m.generate(idx = torch.zeros((1,1), dtype=torch.long), max_new_tokens=300)[0].tolist()))



IOFinge NUCa thofes inomee:
I Ve d tfu bemak o st r, htis g angringhe us ainlyomaroughim ry; geoat yodime istho aund.
Vliolathe'y iowere ESTMUMyene pagginourigafomal areroneee t.
Whingri'd houedlld hecculanoustime d,
Myok, wivea hif h ithethe ho,

Why,
Shomy d myonefitwr, ten!LIO:
CO:
ORD mond ouca


In [129]:
B,T,C = 4, 8, 2
x = torch.randn(B,T,C)
tril = torch.tril(torch.ones((T,T)))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim = -1)
xbow = wei @ x #----> (T, T) @ (B,T,C) == broadcast ==> (B, T, T) @ (B, T, C) -->(B, T, C)
print(x[0])
print(xbow[0])

tensor([[ 0.1771,  1.2758],
        [-0.5472,  0.2220],
        [-0.5466,  1.0820],
        [ 1.8327,  0.6423],
        [ 0.4958,  0.4040],
        [ 0.0217, -0.5402],
        [ 0.9214, -0.2138],
        [ 1.4714,  0.8060]])
tensor([[ 0.1771,  1.2758],
        [-0.1850,  0.7489],
        [-0.3055,  0.8599],
        [ 0.2290,  0.8055],
        [ 0.2824,  0.7252],
        [ 0.2389,  0.5143],
        [ 0.3364,  0.4103],
        [ 0.4783,  0.4598]])


In [135]:
torch.arange(20)

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19])